In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision import datasets
import torchvision.transforms as transforms
from tqdm import tqdm
import math

In [2]:
def squash(input):
  squared_norm=(input**2).sum(dim=-1,keepdim=True)
  output=squared_norm*input/((1.+squared_norm)*torch.sqrt(squared_norm))
  return output

In [3]:
def to_one_hot(x,size):
    x_one_hot=torch.zeros(x.size()[0],size)
    for i in range(x.size()[0]):
        x_one_hot[i,x[i]]=1.0
    return x_one_hot

In [4]:
class Conv(nn.Module):
  def __init__(self,input_num_channels=1,output_num_channels=256,k_size=9):
    super(Conv,self).__init__()
    self.convolution=nn.Conv2d(in_channels=input_num_channels,out_channels=output_num_channels,kernel_size=[k_size,k_size],stride=1,bias=True)
  def forward(self,input):
    #input size is (batch_size, 1, 28, 28)
    output=self.convolution(input)
    output=F.relu(output)
    #output size is (batch_size, 256, 20, 20)
    return output

In [5]:
class Primary_Caps(nn.Module):
  def __init__(self,num_caps=32,input_channels=256,output_channels=8,k_size=9):
    super(Primary_Caps,self).__init__()
    self.primary_capsules=nn.ModuleList([nn.Conv2d(in_channels=input_channels, out_channels=output_channels,kernel_size=[k_size,k_size],stride=2,bias=True)for _ in range(num_caps)])
  def forward(self,input):
    #input size is (batch_size, 256, 20, 20)
    output=[p_c(input) for p_c in self.primary_capsules]
    output=torch.stack(output, dim=1)
    #output size is (batch_size, 32, 8, 6, 6)
    output=torch.transpose(output,1,2)
    #output size is (batch_size, 8, 32, 6, 6)
    output=torch.reshape(output,(batch_size,-1,output.size()[1]))
    #output size is (batch_size, 32*6*6, 8)
    output=squash(output)
    return output

In [6]:
class Digit_Caps(nn.Module):
  def __init__(self,num_caps=10,input_size=8,output_size=16,num_routes=1152):
    super(Digit_Caps,self).__init__()
    self.num_caps=num_caps
    self.num_routes=num_routes
    self.W=nn.Parameter(torch.randn((batch_size,num_routes,num_caps,output_size,input_size)))
  def forward(self,input):
    #input size is (batch_size, 32*6*6, 8)
    #W size is (batch_size, 32*6*6, 10, 16, 8) 
    input=torch.stack([input] * self.num_caps, dim=2).unsqueeze(4)
    #input size is (batch_size, 32*6*6, 10, 8, 1)
    u_hat=torch.matmul(self.W,input)
    #u_hat size is (batch_size, 32*6*6, 10, 16, 1)
    b=Variable(torch.zeros(batch_size,self.num_routes,self.num_caps,1))
    #b size is (batch_size, 32*6*6, 10, 1)
    b=b.cuda()
    for i in range(0,3,1):
      c=F.softmax(b,dim=1).unsqueeze(4)
      #c size is (batch_size, 32*6*6, 10, 1, 1)
      s=c*u_hat
      #s size is (batch_size, 32*6*6, 10, 16, 1)
      s=s.sum(dim=1 ,keepdim=True)
      #s size is (batch_size, 1, 10, 16, 1)
      v=squash(s)
      #v size is (batch_size, 1, 10, 16, 1)
      if i<2:
        a=torch.matmul(u_hat.transpose(3, 4), torch.cat([v] * self.num_routes, dim=1))
        #a size is (batch_size, 32*6*6, 10, 1, 1)
        b=b+a.squeeze(4).mean(dim=0, keepdim=True)
        #b size is (batch_size, 32*6*6, 10, 1)
    return v.squeeze(1)

In [7]:
class Decoder(nn.Module):
  def __init__(self,input_size=160,decoder1_size=512,decoder2_size=1024,output_size=784,img_height=28,img_width=28,img_channels=1):
    super(Decoder,self).__init__()
    self.num_channels=img_channels
    self.height=img_height
    self.width=img_width
    self.linear1=nn.Linear(in_features=input_size,out_features=decoder1_size,bias=True)
    self.linear2=nn.Linear(in_features=decoder1_size,out_features=decoder2_size,bias=True)
    self.linear3=nn.Linear(in_features=decoder2_size,out_features=output_size,bias=True)
  def forward(self,input):
    #input size is (batch_size, 10, 16, 1)
    classes=torch.sqrt((input**2).sum(2))
    classes=F.softmax(classes, dim=0)
    #classes size is (batch_size, 10, 1)
    _,max_length_indices=classes.max(dim=1)
    #max_length_indices size is (batch_size, 1)
    masked=Variable(torch.sparse.torch.eye(10))
    masked=masked.cuda()
    masked=masked.index_select(dim=0,index=Variable(max_length_indices.squeeze(1).data))
    #masked size is (batch_size, 10)
    input=(input*masked[:,:,None,None]).view(input.size(0),-1)
    #input size is (batch_size, 160)
    output=self.linear1(input)
    output=F.relu(output)
    #output size is (batch_size, 512)
    output=self.linear2(output)
    output=F.relu(output)
    #output size is (batch_size, 1024)
    output=self.linear3(output)
    output=F.relu(output)
    #output size is (batch_size, 784)
    output=torch.reshape(output,(batch_size,self.num_channels,self.height,self.width))
    #output size is (batch_size, 1, 28, 28)
    return output, masked

In [8]:
class CapsNet(nn.Module):
  def __init__(self):
    super(CapsNet,self).__init__()
    self.conv=Conv()
    self.p_cap=Primary_Caps()
    self.d_cap=Digit_Caps()
    self.dec=Decoder()
  def forward(self,input):
    out_conv=self.conv(input)
    out_p_c=self.p_cap(out_conv)
    output=self.d_cap(out_p_c)
    rec,mask=self.dec(output)
    return output,rec,mask
  def margin_loss(self,output,true_output):
    Lambda=0.5
    m_plus=0.9
    m_minus=0.1
    squared_norm=(output**2).sum(2,keepdim=True)
    norm=torch.sqrt(squared_norm)
    term1=F.relu(m_plus-norm).view(batch_size,-1)
    term2=F.relu(norm-m_minus).view(batch_size,-1)
    loss=true_output*term1+Lambda*(1.0-true_output)*term2
    loss=loss.sum(-1).mean()
    return loss
  def reconstruction_loss(self,input,rec):
    mse=nn.MSELoss()
    input=torch.reshape(input,(input.size()[0],-1))
    rec=torch.reshape(rec,(rec.size()[0],-1))
    loss=0.005*mse(input,rec)
    return loss
  def loss(self,input,output,true_output,rec):
    loss_m_l=self.margin_loss(output,true_output)
    loss_r_l=self.reconstruction_loss(input,rec)
    total_loss=loss_m_l+loss_r_l
    return total_loss

In [9]:
def train(capsnet,optimizer,train_loader,epoch,save_path='capsnet'):
    capsnet.train()
    num_batches=len(list(enumerate(train_loader)))
    total_loss=0
    for batch_idx,(input,label) in enumerate(tqdm(train_loader)):
        target=torch.sparse.torch.eye(10).index_select(dim=0,index=label)
        input,target=Variable(input),Variable(target)
        input,target=input.cuda(),target.cuda()
        output,rec,masked=capsnet(input)
        loss=capsnet.loss(input,output,target,rec)
        train_loss=loss.item()
        if math.isnan(train_loss):
          capsnet=torch.load(save_path)
        else:
          loss.backward()
          optimizer.step()
          total_loss+=train_loss
          #torch.save(capsnet,save_path)
        correct=sum(np.argmax(masked.data.cpu().numpy(),1)==np.argmax(target.data.cpu().numpy(),1))
        #if math.isnan(train_loss):
          #print("loading previous best model")
          #capsnet=torch.load(save_path)
        if batch_idx%100==0:
            optimizer.zero_grad()
            tqdm.write("Epoch: [{}/{}], Batch: [{}/{}], train accuracy: {:.6f}, loss: {:.6f}".format(
                epoch,num_epochs,batch_idx+1,num_batches,correct/float(batch_size),train_loss/float(batch_size)))
            if not (math.isnan(train_loss)):
              torch.save(capsnet,save_path)
    tqdm.write('Epoch: [{}/{}], train loss: {:.6f}'.format(epoch,num_epochs,total_loss/len(train_loader.dataset)))
    #if not (math.isnan(total_loss)):
      #torch.save(capsnet,save_path)

In [10]:
def test(capsnet,test_loader,epoch):
    capsnet
    capsnet.eval()
    test_loss=0
    correct=0
    for batch_idx,(input,label) in enumerate(test_loader):
        target=torch.sparse.torch.eye(10).index_select(dim=0,index=label)
        input,target=Variable(input),Variable(target)
        input,target=input.cuda(),target.cuda()
        output,rec,masked=capsnet(input)
        loss=capsnet.loss(input,output,target,rec)
        test_loss+=loss.item()
        correct+=sum(np.argmax(masked.data.cpu().numpy(),1)==np.argmax(target.data.cpu().numpy(),1))
    tqdm.write("Epoch: [{}/{}], test accuracy: {:.6f}, loss: {:.6f}".format(epoch,num_epochs,correct/len(test_loader.dataset),test_loss/len(test_loader)))

In [11]:
data_transform = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize((0.1307,),(0.3081,))
                ])
batch_size=128
num_epochs=30
capsnet=CapsNet()
capsnet=torch.nn.DataParallel(capsnet)
train_loader=torch.utils.data.DataLoader(datasets.MNIST("mnist",train=True,download=True,transform=data_transform),batch_size=batch_size,shuffle=True,drop_last=True)
test_loader=torch.utils.data.DataLoader(datasets.MNIST("mnist",train=False,download=True,transform=data_transform),batch_size=batch_size,shuffle=True,drop_last=True)
capsnet=capsnet.cuda()
capsnet=capsnet.module
optimizer=torch.optim.Adam(capsnet.parameters(),lr=1e-3)
for e in range(1,num_epochs+1):
  train(capsnet,optimizer,train_loader,e)
  test(capsnet,test_loader,e)
  torch.cuda.empty_cache()

  0%|          | 0/468 [00:00<?, ?it/s]

Epoch: [1/30], Batch: [1/468], train accuracy: 0.125000, loss: 0.007060


 21%|██▏       | 100/468 [00:55<03:03,  2.00it/s]

Epoch: [1/30], Batch: [101/468], train accuracy: 0.117188, loss: 0.006719


 43%|████▎     | 200/468 [01:49<02:15,  1.97it/s]

...

Epoch: [30/30], Batch: [301/468], train accuracy: 0.976562, loss: 0.000431


 85%|████████▌ | 400/468 [03:45<00:35,  1.93it/s]

Epoch: [30/30], Batch: [401/468], train accuracy: 0.992188, loss: 0.000359


100%|██████████| 468/468 [04:23<00:00,  1.77it/s]


Epoch: [30/30], train loss: nan
Epoch: [30/30], test accuracy: 0.980900, loss: 0.060096
